tgb - 4/15/2020
- Adapting Ankitesh's notebook that builds and train a "brute-force" network to David Walling's hyperparameter search  
- Adding the option to choose between aquaplanet and real-geography data

In [ ]:
import sys
sys.path.insert(1,"/home1/07064/tg863631/anaconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages") #work around for h5py
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
from cbrain.climate_invariant import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
# from climate_invariant import *
from tensorflow.keras import layers
import datetime
from climate_invariant_utils import *
import yaml


## Global Variables

In [ ]:
# Load coordinates (just pick any file from the climate model run)

# Comet path below
# coor = xr.open_dataset("/oasis/scratch/comet/ankitesh/temp_project/data/sp8fbp_minus4k.cam2.h1.0000-01-01-00000.nc",\
#                     decode_times=False)

# GP path below
path_0K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/'
coor = xr.open_dataset(path_0K+"AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-09-02-00000.nc")

lat = coor.lat; lon = coor.lon; lev = coor.lev;
coor.close();

In [ ]:
# Comet path below
# TRAINDIR = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/CRHData/'
# path = '/home/ankitesh/CBrain_project/CBRAIN-CAM/cbrain/'

# GP path below
TRAINDIR = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path = '/export/nfs0home/tbeucler/CBRAIN-CAM/cbrain/'
path_nnconfig = '/export/nfs0home/tbeucler/CBRAIN-CAM/nn_config/'

# Load hyam and hybm to calculate pressure field in SPCAM
path_hyam = 'hyam_hybm.pkl'
hf = open(path+path_hyam,'rb')
hyam,hybm = pickle.load(hf)

# Scale dictionary to convert the loss to W/m2
scale_dict = load_pickle(path_nnconfig+'scale_dicts/009_Wm2_scaling.pkl')

New Data generator class for the climate-invariant network. Calculates the physical rescalings needed to make the NN climate-invariant

In [ ]:
class DataGeneratorClimInv(DataGenerator):
    
    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,                 
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train',portion=1):
        
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        if rh_trans:
            self.qv2rhLayer = QV2RHNumpy(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)
        
        if lhflx_trans:
            self.lhflxLayer = LhflxTransNumpy(self.inp_sub,self.inp_div,hyam,hybm)
            
        if t2tns_trans:
            self.t2tnsLayer = T2TmTNSNumpy(self.inp_sub,self.inp_div,inp_subTNS,inp_divTNS,hyam,hybm)
            
        if scaling:
            self.scalingLayer = ScalingNumpy(hyam,hybm)
            self.inp_shape += 1
                    
        if interpolate:
            self.interpLayer = InterpolationNumpy(lev,is_continous,Tnot,lower_lim,interm_size)
            self.inp_shape += interm_size*2 + 4 + 30 ## 4 same as 60-64 and 30 for lev_tilde.size
            
        # tgb - 7/9/2020 - Test only training on a subset of the data determined by portion
        self.portion = portion
            
        
    def __getitem__(self, index):
        
        # If portion<1, only look at a subset of the data by putting an upper bound on index
        if self.portion<1: index = index % round(1/self.portion)
        elif self.portion>1: print('Setting portion=1 because portion>1')
        elif self.portion<0: print('Setting portion=1 because portion<0')
        
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        X_result = X_norm
        
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result) 
            
        if self.lhflx_trans:
            X_result = self.lhflxLayer.process(X_result)
            X_result = X_result[:,:64]
            X = X[:,:64]
            
        if self.t2tns_trans:
            X_result = self.t2tnsLayer.process(X_result)
        
        if self.scaling:
            scalings = self.scalingLayer.process(X) 
            X_result = np.hstack((X_result,scalings))
        
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
            

        if self.mode=='val':
            return xr.DataArray(X_result), xr.DataArray(Y)
        return X_result,Y
    
    ##transforms the input data into the required format, take the unnormalized dataset
    def transform(self,X):
        X_norm = self.input_transform.transform(X)
        X_result = X_norm
        
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result)  
        
        if self.lhflx_trans:
            X_result = self.lhflxLayer.process(X_result)
            X_result = X_result[:,:64]
            X = X[:,:64]

        if self.t2tns_trans:
            X_result = self.t2tnsLayer.process(X_result)
        
        if self.scaling:
            scalings = self.scalingLayer.process(X) 
            X_result = np.hstack((X_result,scalings))
        
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
            

        return X_result

## Data Generators

### Choose between aquaplanet and realistic geography here

In [4]:
# GP paths below
path_aquaplanet = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path_realgeography = ''

# Comet paths below
# path_aquaplanet = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'
# path_realgeography = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/geography/'

path = path_aquaplanet

### Baseline Data Generator, without ozone as input but with radiative outputs

In [5]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','QRL', 'QRS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_O3 = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE_O3 = '2021_01_24_NORM_O3_small.nc'
VALIDFILE_O3 = '2021_01_24_O3_VALID.nc'
TESTFILE_O3 = '2021_01_24_O3_TEST.nc'

In [6]:
train_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [7]:
valid_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+VALIDFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [8]:
test_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+TESTFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

### Data Generator using RH

In [ ]:
#scale_dict_RH = load_pickle('/home/ankitesh/CBrain_project/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling_2.pkl')
scale_dict_RH = scale_dict.copy()
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%

in_vars_RH = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','QRL','QRS']

# New GP path below
TRAINFILE_RH = '2021_01_24_O3_small_shuffle.nc'
NORMFILE_RH = '2021_02_01_NORM_O3_RH_small.nc'
    
# Comet/Ankitesh path below
# TRAINFILE_RH = 'CI_RH_M4K_NORM_train_shuffle.nc'
# NORMFILE_RH = 'CI_RH_M4K_NORM_norm.nc'
# VALIDFILE_RH = 'CI_RH_M4K_NORM_valid.nc'

In [ ]:
train_gen_RH = DataGenerator(
    data_fn = path+TRAINFILE_RH,
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = path+NORMFILE_RH,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict_RH,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using TNS

In [ ]:
in_vars = ['QBP','TfromNS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_TNS = 'CI_TNS_M4K_NORM_train_shuffle.nc'
NORMFILE_TNS = 'CI_TNS_M4K_NORM_norm.nc'
VALIDFILE_TNS = 'CI_TNS_M4K_NORM_valid.nc'

In [ ]:
train_gen_TNS = DataGenerator(
    data_fn = path+TRAINFILE_TNS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_TNS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using $O_{3}$

In [ ]:
in_vars = ['QBP','TBP','O3_AQUA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','QRL', 'QRS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_O3 = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE_O3 = '2021_01_24_NORM_O3_small.nc'
VALIDFILE_O3 = '2021_01_24_O3_VALID.nc'
TESTFILE_O3 = '2021_01_24_O3_TEST.nc'

In [ ]:
train_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [ ]:
valid_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+VALIDFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [ ]:
test_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+TESTFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

### Data Generator Combined (latest flexible version)

In [10]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
# elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
if path==path_aquaplanet: out_vars=['PHQ','TPHYSTND','QRL','QRS']

In [11]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

TRAINFILE = '2021_03_18_O3_TRAIN_M4K.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE = '2021_03_18_O3_TEST_P4K.nc'

Old data generator by Ankitesh

In [12]:
# TRAINFILE = 'CI_SP_M4K_train_shuffle.nc'
# NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
# VALIDFILE = 'CI_SP_M4K_valid.nc'


# train_gen_CI = DataGeneratorClimInv(
#     data_fn = path+TRAINFILE,
#     input_vars = in_vars,
#     output_vars = out_vars,
#     norm_fn = path+NORMFILE,
#     input_transform = ('mean', 'maxrs'),
#     output_transform = scale_dict,
#     batch_size=8192,
#     shuffle=True,
#     lev=lev,
#     hyam=hyam,hybm=hybm,
#     inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
#     inp_subTNS=train_gen_RH.input_transform.sub,inp_divTNS=train_gen_RH.input_transform.div,
#     rh_trans=True,t2tns_trans=False,
#     lhflx_trans=False,
#     scaling=False,
#     interpolate=False
# )

# valid_gen_CI = DataGeneratorClimInv(
#     data_fn = path+VALIDFILE,
#     input_vars = in_vars,
#     output_vars = out_vars,
#     norm_fn = path+NORMFILE,
#     input_transform = ('mean', 'maxrs'),
#     output_transform = scale_dict,
#     batch_size=8192,
#     shuffle=True,
#     lev=lev,
#     hyam=hyam,hybm=hybm,
#     inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
#     inp_subTNS=train_gen_RH.input_transform.sub,inp_divTNS=train_gen_RH.input_transform.div,
#     rh_trans=True,t2tns_trans=False,
#     lhflx_trans=False,
#     scaling=False,
#     interpolate=False
# )


Improved flexible data generator

In [13]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [14]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [ ]:
test_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

### Add callback class to track loss on multiple sets during training

From [https://stackoverflow.com/questions/47731935/using-multiple-validation-sets-with-keras]

In [19]:
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        for validation_set in self.validation_sets:
            if len(validation_set) not in [3, 4]:
                raise ValueError()
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            if len(validation_set) == 3:
                validation_data, validation_targets, validation_set_name = validation_set
                sample_weights = None
            elif len(validation_set) == 4:
                validation_data, validation_targets, sample_weights, validation_set_name = validation_set
            else:
                raise ValueError()

            results = self.model.evaluate(x=validation_data,
                                          y=validation_targets,
                                          verbose=self.verbose,
                                          sample_weight=sample_weights,
                                          batch_size=self.batch_size)

            for metric, result in zip(self.model.metrics_names,results):
                valuename = validation_set_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

## Brute-Force Model

### Climate-invariant (T,Q,PS,S0,SHF,LHF)->($\dot{T}$,$\dot{q}$,RADFLUX)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(64, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_temp'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

### Ozone (T,Q,$O_{3}$,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(94,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_O3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

### No Ozone (T,Q,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_noO3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
# Nep = 15
# model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
#               callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
              callbacks=[earlyStopping, mcp_save_pos])

### BF linear version

In [15]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [18]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_18_MLR'

In [ ]:
history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

### RH linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_02_01_MLR_RH'

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos])

### TfromNS linear version

### Bcons linear version

## Validation Check

### Chech on validation dataset (m4K simulation) to make sure the NN was properly fitted

In [ ]:
Ankitesh_models = '/oasis/scratch/comet/ankitesh/temp_project/models/'
path_aqua = 'BF.hdf5'
path_geo = 'BF_Geography.hdf5'

In [ ]:
model_aqua = load_model(Ankitesh_models+path_aqua)
model_geo = load_model(Ankitesh_models+path_geo)

tgb - 7/8/2020 - Test loading `pb` model

### TODO

In [ ]:
config_file = 'CI_SP_M4K_CONFIG.yml' # Configuration file
if path==path_aquaplanet: data_file = ['CI_SP_M4K_valid.nc','CI_SP_P4K_valid.nc']  # Validation/test data sets
elif path==path_realgeography: data_file = ['geography/CI_SP_M4K_valid.nc','geography/CI_SP_P4K_valid.nc']
NNarray = [path_aqua,path_geo]
NNname = ['Fit_aqua','Fit_geo'] # Name of NNs for plotting

In [ ]:
NN = {}; md = {};
for i,NNs in enumerate(NNarray):
    print('NN name is ',NNs)
    md[NNs] = {}
    for j,data in enumerate(data_file):
        print('data name is ',data)
        
        NN[NNs] = load_model(Ankitesh_models+NNs)
        md[NNs][data[6:-3]] = ModelDiagnostics(NN[NNs],
                                                '/home/ankitesh/CBrain_project/PrepData/'+config_file,
                                                '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'+data)

In [ ]:
data_file

In [ ]:
md

In [ ]:
data_file

In [ ]:
data = data_file[1]

In [ ]:
md[NNs][data[6:-3]].valid_gen.n_batches

### Diagnostics at random drawn time steps

In [ ]:
NNarray = ['2021_01_25_O3','2021_01_25_noO3']
#dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

In [ ]:
path = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'

In [ ]:
NN = {};
for i,NNs in enumerate(NNarray):
    print('NN name is ',NNs+'.hdf5')
    path_model = path+NNs+'.hdf5'
    NN[NNs] = load_model(path_model)

In [ ]:
gen = valid_gen_O3

$O_{3}$ vs no $O_{3}$

In [ ]:
Nt = 100
t_random = np.random.choice(np.linspace(0,1691,1692),size=((Nt,)),replace=False).astype('int')

In [ ]:
# Direct
MSE = {}
VAR = {}
INP = {}

for iar,itime in enumerate(t_random):
    print('iar=',iar,'/',Nt-1,' & itime',itime,end="\r")
    for i,NNs in enumerate(NNarray):
        if iar==0: MSE[NNs] = {}; VAR[NNs] = {}
        
        inp = gen[itime][0]
        inp_noO3 = np.delete(inp,np.arange(60,90),axis=1)
        truth = gen[itime][1]
        
        if i==0: p = NN[NNs].predict_on_batch(inp)
        elif i==1: p = NN[NNs].predict_on_batch(inp_noO3)
        
        inp_geo = np.reshape(inp,(64,128,inp.shape[1],1))
        p_geo = np.reshape(p,(64,128,p.shape[1]))
        t_geo = np.reshape(truth,(64,128,truth.shape[1]))
        
        if iar==0: 
            MSE[NNs] = np.mean((t_geo-p_geo)**2,axis=2)
            VAR[NNs] = np.var(p_geo,axis=2)
            INP[NNs] = inp_geo
        else: 
            MSE[NNs] = np.concatenate((MSE[NNs],np.mean((t_geo-p_geo)**2,axis=2)),axis=1)
            VAR[NNs] = np.concatenate((VAR[NNs],np.var(p_geo,axis=2)),axis=1)
            INP[NNs] = np.concatenate((INP[NNs],inp_geo),axis=3)

In [ ]:
INP[NNs].shape

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,89,:],axis=(1,2)),label='z=29')
plt.plot(np.mean(INP[NNs][:,:,79,:],axis=(1,2)),label='z=19')
plt.plot(np.mean(INP[NNs][:,:,69,:],axis=(1,2)),label='z=9')
plt.legend()
plt.xlabel('Latitude')
plt.ylabel('Norm O3 value')

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,89,:],axis=(0,2)),label='29')
plt.plot(np.mean(INP[NNs][:,:,79,:],axis=(0,2)),label='19')
plt.plot(np.mean(INP[NNs][:,:,69,:],axis=(0,2)),label='9')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Norm O3 value')

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,60:90,:],axis=(0,1,3))*\
         train_gen_O3.input_transform.div[60:90]+\
         train_gen_O3.input_transform.sub[60:90])
plt.ylabel('True O3 value')
plt.xlabel('Vertical level')

In [ ]:
inp.div

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,60:90,:],axis=(0,1,3)))

In [ ]:
MSE['2021_01_25_O3'].mean()

In [ ]:
MSE['2021_01_25_noO3'].mean()

In [ ]:
p.shape

In [ ]:
# Using md
MSE = {}
VAR = {}
diagno = {}
diagno['truth'] = {}
diagno['pred'] = {}

for iar,itime in enumerate(t_random):
    print('iar=',iar,'/',Nt-1,' & itime',itime,end="\r")
    for i,NNs in enumerate(NNarray):
        if iar==0: MSE[NNs] = {}; VAR[NNs] = {}

        inp, p, truth = md[NNs][data[6:-3]].get_inp_pred_truth(itime)  # [lat, lon, var, lev]
        
        t_geo = md[NNs][data[6:-3]].reshape_ngeo(truth)[:,:,:]
        p_geo = md[NNs][data[6:-3]].reshape_ngeo(p.numpy())[:,:,:]
        
        if iar==0: 
            MSE[NNs][data[6:-3]] = np.mean((t_geo-p_geo)**2,axis=2)
            VAR[NNs][data[6:-3]] = np.var(p_geo,axis=2)
        else: 
            MSE[NNs][data[6:-3]] = np.concatenate((MSE[NNs][data[6:-3]],
                                                   np.mean((t_geo-p_geo)**2,axis=2)),axis=1)
            VAR[NNs][data[6:-3]] = np.concatenate((VAR[NNs][data[6:-3]],
                                                   np.var(p_geo,axis=2)),axis=1)

#         MSE[NNs] = (MSE['samples']*MSE[NNs]+np.mean((p-truth)**2))/(MSE['samples']+1)
#         MSE['samples'] += 1

In [ ]:
for i,NNs in enumerate(NNarray):
    plt.scatter(np.mean(coor.TS,axis=(0,2)),np.log10(np.mean(MSE[NNs][data[6:-3]],axis=1)),label=NNs)
plt.legend()
plt.title(data[6:-3])

In [ ]:
for i,NNs in enumerate(NNarray):
    plt.scatter(np.mean(coor.TS,axis=(0,2)),np.mean(MSE[NNs][data[6:-3]],axis=1)/
                np.mean(VAR[NNs][data[6:-3]],axis=1),label=NNs)
plt.legend()
plt.title(data[6:-3])

In [ ]:
coor.TS.shape

In [ ]:
model_aqua.evaluate_generator(valid_gen,steps=100)

In [ ]:
model_geo.evaluate_generator(valid_gen,steps=100)

In [ ]:
model = load_model(path_HDF5+save_name+'.hdf5')
model.evaluate_generator(valid_gen)

### Check on test dataset (p4k simulation) to test ability of NN to generalize to unseen climate

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']

In [ ]:
NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
VALIDFILE = 'CI_SP_P4K_valid.nc'

valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)

In [ ]:
model = load_model(path_HDF5+save_name+'.hdf5')
model.evaluate_generator(valid_gen)

## Retrain BF models for transfer learning

In [ ]:
NN

In [ ]:
BF_geog = NN['BF_Geography.hdf5']

In [ ]:
BF_geog.summary()

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_Geog_2020_07_22'

In [ ]:
#model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
BF_geog.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

## Transfer Learning experiments

tgb - 7/9/2020 - Use portion<1

### From BF aqua to BF geo

In [ ]:
TRAINFILE = 'CI_SP_M4K_train_shuffle.nc'
NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
VALIDFILE = 'CI_SP_M4K_valid.nc'

tgb - 7/23/2020 - Only training last three indices, be careful with saved names!

In [ ]:
por_array = [0.01,0.1,1]

In [ ]:
Nep = 10

In [ ]:
NN = {}
for i,por in enumerate(por_array):
    print('por=',por)
    graph = tf.Graph()
    
    #with tf.Session(graph=graph): # Legacy from tf1
        
    # 1) Define new generators
    train_gen = DataGeneratorClimInv(
    data_fn = path+TRAINFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    portion=por
    )

    valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    )

    # 2) Load model
    NN[por] = load_model('/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/BF_Aqua_2020_07_22.hdf5') # Load aquaplanet model

    # 3) Define callbacks and save_name of new model
    path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
    save_name = 'TL_BF_2020_07_23_porindex_'+str(i+3)
    earlyStopping = EarlyStopping(monitor='loss', patience=5, verbose=0, mode='min')
    mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='loss', mode='min')

    # 4) Train model for Nep epochs and CANNOT save state of best validation loss because
    # it would NOT be consistent with transfer learning scenario
    NN[por].fit_generator(train_gen, epochs=Nep, callbacks=[earlyStopping, mcp_save_pos])

tgb - 7/23/2020 - Legacy = Trained por_array = `[0.00001,0.0001,0.001]` already and now only training the last three indices

In [ ]:
por_array = [0.00001,0.0001,0.001,0.01,0.1,1]

In [ ]:
por_array

In [ ]:
Nep = 10

In [ ]:
NN = {}
for i,por in enumerate(por_array):
    print('por=',por)
    graph = tf.Graph()
    
    #with tf.Session(graph=graph): # Legacy from tf1
        
    # 1) Define new generators
    train_gen = DataGeneratorClimInv(
    data_fn = path+TRAINFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    portion=por
    )

    valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    )

    # 2) Load model
    NN[por] = load_model('/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/BF_Aqua_2020_07_22.hdf5') # Load aquaplanet model

    # 3) Define callbacks and save_name of new model
    path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
    save_name = 'TL_BF_2020_07_23_porindex_'+str(i)
    earlyStopping = EarlyStopping(monitor='loss', patience=5, verbose=0, mode='min')
    mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='loss', mode='min')

    # 4) Train model for Nep epochs and CANNOT save state of best validation loss because
    # it would NOT be consistent with transfer learning scenario
    NN[por].fit_generator(train_gen, epochs=Nep, callbacks=[earlyStopping, mcp_save_pos])